In [ ]:
!git clone https://github.com/deepmind/alphamissense.git
%cd alphamissense


Cloning into 'alphamissense'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 152 (delta 70), reused 146 (delta 64), pack-reused 0 (from 0)
Receiving objects: 100% (152/152), 197.49 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (70/70), done.
/content/alphamissense


In [ ]:
# Install system dependencies
!apt-get update
!apt-get install -y aria2 hmmer

# Install Python dependencies
!pip install -r requirements.txt
!pip install -e .


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,616 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

Obtaining file:///content/alphamissense
  Preparing metadata (setup.py) ... done
  Running setup.py develop for alphamissense


In [ ]:
!python test/test_installation.py


Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.10/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 85, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'
Traceback (most recent call last):
  File "/usr/lib/python3.10/subprocess.py", line 1154, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/usr/lib/python3.10/subprocess.py", line 2021, in _communicate
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurr

In [ ]:
# Example: Download genetic databases or other files
!aria2c -x 16 -s 16 <URL_TO_GENETIC_DATABASE>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
import requests

# Define download URLs
DATABASES = {
    "UniRef90": "https://storage.googleapis.com/alphafold/alphafold_data/uniref90/uniref90.fasta",
    "MGnify": "https://storage.googleapis.com/alphafold/alphafold_data/mgnify/mgy_clusters_2022_05.fa",
    "Small BFD": "https://storage.googleapis.com/alphafold/alphafold_data/small_bfd/bfd-first_non_consensus_sequences.fasta",
}

# Define local paths for storing databases
BASE_DIR = "/content/databases"
os.makedirs(BASE_DIR, exist_ok=True)

def download_file(url, save_path):
    """Download a file from a URL to a local directory."""
    print(f"Downloading {url} to {save_path}...")
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded: {save_path}")
    else:
        print(f"Failed to download {url}. Status code: {response.status_code}")
        response.raise_for_status()

# Download all databases
for db_name, url in DATABASES.items():
    db_path = os.path.join(BASE_DIR, db_name.replace(" ", "_").lower() + ".fasta")
    if not os.path.exists(db_path):
        download_file(url, db_path)
    else:
        print(f"{db_name} already exists at {db_path}")

# Assign paths to variables
uniref90_path = os.path.join(BASE_DIR, "uniref90.fasta")
mgnify_path = os.path.join(BASE_DIR, "mgy_clusters_2022_05.fa")
small_bfd_path = os.path.join(BASE_DIR, "bfd-first_non_consensus_sequences.fasta")

# Set up paths for the AlphaMissense pipeline
from alphamissense.data import pipeline_missense

protein_sequence_file = "/path/to/your/fasta_file.fasta"
jackhmmer_binary_path = "/usr/bin/jackhmmer"  # Update if needed
msa_output_dir = "/content/msa_output"
os.makedirs(msa_output_dir, exist_ok=True)

pipeline = pipeline_missense.DataPipeline(
    jackhmmer_binary_path=jackhmmer_binary_path,
    protein_sequence_file=protein_sequence_file,
    uniref90_database_path=uniref90_path,
    mgnify_database_path=mgnify_path,
    small_bfd_database_path=small_bfd_path,
)

# Test the pipeline with a sample input
sample = pipeline.process(
    protein_id="SampleProtein",  # Replace with actual protein ID in your FASTA
    reference_aa="A",
    alternate_aa="V",
    position=123,  # 1-based position
    msa_output_dir=msa_output_dir,
)

print("Pipeline setup completed!")


Failed to download https://storage.googleapis.com/alphafold/alphafold_data/uniref90/uniref90.fasta. Status code: 403


HTTPError: 403 Client Error: Forbidden for url: https://storage.googleapis.com/alphafold/alphafold_data/uniref90/uniref90.fasta